In [1]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.layers import Dense,Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout

from keras.utils import np_utils
from tensorflow.python.keras.utils.layer_utils import print_summary
from tensorflow.keras.layers import Layer

import tensorflow as tf
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import pickle
from keras.callbacks import TensorBoard

def keras_model(image_x, image_y):
    # npy 파일 즉, 컴퓨터가 학습할 파일의 개수를 의미
    num_of_classes = 2
    
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(image_x,image_y,1), activation='relu'))
    model.add(Conv2D(32, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(Dropout(0.25))
    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_of_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    filepath = "QuickDraw.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    return model, callbacks_list


def loadFromPickle():
    with open("features", "rb") as f:
        features = np.array(pickle.load(f))
    with open("labels", "rb") as f:
        labels = np.array(pickle.load(f))

    return features, labels


def augmentData(features, labels):
    features = np.append(features, features[:, :, ::-1], axis=0)
    labels = np.append(labels, -labels, axis=0)
    return features, labels


def prepress_labels(labels):
    labels = np_utils.to_categorical(labels)
    return labels


def main():
    features, labels = loadFromPickle()
    # features, labels = augmentData(features, labels)
    features, labels = shuffle(features, labels)
    labels=prepress_labels(labels)
    train_x, test_x, train_y, test_y = train_test_split(features, labels, random_state=0,
                                                        test_size=0.1)
    train_x = train_x.reshape(train_x.shape[0], 28, 28, 1)
    test_x = test_x.reshape(test_x.shape[0], 28, 28, 1)
    model, callbacks_list = keras_model(28,28)
    print_summary(model)
    model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=50, batch_size=64,
              callbacks=[TensorBoard(log_dir="QuickDraw")])
    model.save('QuickDraw.h5')


main()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        25632     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 10, 10, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 64)          51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 64)          0

282/282 [==============================] - 11s 39ms/step - loss: 0.0037 - accuracy: 0.9989 - val_loss: 0.0603 - val_accuracy: 0.9900
Epoch 44/50
282/282 [==============================] - 11s 39ms/step - loss: 0.0016 - accuracy: 0.9995 - val_loss: 0.0829 - val_accuracy: 0.9895
Epoch 45/50
282/282 [==============================] - 11s 39ms/step - loss: 0.0053 - accuracy: 0.9985 - val_loss: 0.0404 - val_accuracy: 0.9875
Epoch 46/50
282/282 [==============================] - 11s 39ms/step - loss: 0.0018 - accuracy: 0.9996 - val_loss: 0.0660 - val_accuracy: 0.9900
Epoch 47/50
282/282 [==============================] - 11s 39ms/step - loss: 0.0012 - accuracy: 0.9997 - val_loss: 0.0673 - val_accuracy: 0.9890
Epoch 48/50
282/282 [==============================] - 11s 39ms/step - loss: 0.0029 - accuracy: 0.9989 - val_loss: 0.0307 - val_accuracy: 0.9910
Epoch 49/50
282/282 [==============================] - 11s 39ms/step - loss: 0.0020 - accuracy: 0.9995 - val_loss: 0.0555 - val_accuracy: 0.99